In [7]:
import opendatasets as od
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [8]:
url = "https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package"
od.download(url)

Skipping, found downloaded files in ".\weather-dataset-rattle-package" (use force=True to force download)


In [9]:
os.listdir("weather-dataset-rattle-package")

['weatherAUS.csv']

In [10]:
raw_df = pd.read_csv("weather-dataset-rattle-package/weatherAUS.csv")

In [11]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [16]:
raw_df.dropna(subset=["RainTomorrow"], inplace=True)

In [22]:
input_cols = list(raw_df.columns)[1:-1]
input_col = raw_df[input_cols].copy()
target_cols = "RainTomorrow"

In [23]:
numeric_cols = raw_df.select_dtypes(include="number").columns.tolist()
cat_cols = input_col.select_dtypes(object).columns.tolist()

In [27]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="mean").fit(raw_df[numeric_cols])

In [30]:
input_col[numeric_cols] = imputer.transform(input_col[numeric_cols])

In [32]:
from sklearn.preprocessing import MinMaxScaler

In [34]:
scaler = MinMaxScaler().fit(raw_df[numeric_cols])

In [36]:
input_col[numeric_cols] = scaler.transform(input_col[numeric_cols])

In [39]:
input_col.describe().loc[["min","max"]]

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
min,0.200472,0.090737,0.000000,0.000000,0.000000,-0.046512,0.000000,0.000000,0.00,0.00,-16.206612,-15.6336,0.000000,0.000000,0.151899,0.103647
max,0.224057,0.109641,0.002695,0.006897,0.068966,-0.038760,0.007692,0.011494,0.01,0.01,-16.190083,-15.6176,0.111111,0.111111,0.172996,0.122841


In [41]:
from sklearn.preprocessing import OneHotEncoder

In [43]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore").fit(raw_df[numeric_cols])